In [ ]:
import os
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import json
from sklearn.metrics import silhouette_score


def labels_10_vers_5(labels): 

    # 0 no data 
    # 1 artificial surfaces 
    # 2 water bodies 
    # 3 cultivated 
    # 4 braodleaf tree cover and others 

    for i in range(len(labels)): 
        x = labels[i] 
        if x == 0 or x == 1  : 
            labels[i] = 0 
        elif x == 2  or x == 7 :
            labels[i] = 1
        elif x == 9 or x == 8 : 
            labels[i] = 2
        elif x == 3 :
            labels[i] = 3 
        else : 
            labels[i] = 4

    return labels


def kmeans():
    folder_path        = 'dataset/train/images'
    labels_folder_path = 'dataset/train/masks'

    n = '23017.tif'
    
        
    DATA = np.array(tiff.imread(os.path.join(folder_path, n))).reshape(-1,4)

    n_clusters_list = range(2,10)
    silhouette_scores = []
    for n_clusters in n_clusters_list:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        labels = kmeans.fit_predict(DATA)
        score = silhouette_score(DATA, labels)
        silhouette_scores.append(score)

    optimal_n_clusters = n_clusters_list[np.argmax(silhouette_scores)]

    kmeans = KMeans(n_clusters=optimal_n_clusters, random_state=0)
    kmeans.fit(DATA)
    LABELS = kmeans.labels_
    LABELS = LABELS.reshape(256,256,1)


    true_labels = np.array(tiff.imread(os.path.join(labels_folder_path, n)))
    img = np.array(tiff.imread(os.path.join(folder_path, n)))[:,:,:3]
    img = (img / np.max(img))

    plt.subplot(1,3,1)
    plt.imshow(LABELS)
    plt.title("classification par IA, k="+str(optimal_n_clusters))

    plt.subplot(1,3,2)
    plt.imshow(true_labels)
    plt.title("mask")

    plt.subplot(1,3,3) 
    plt.imshow(img)
    plt.title("image réelle")
    plt.suptitle(" Algorithme des k moyennes ")
    plt.show()
          

kmeans()

